In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

## URL von allen Hauptseiten holen

In [ ]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"GeschäftsNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
pd.DataFrame(alle_seiten)

In [ ]:
df_a=pd.DataFrame(alle_seiten)

### Eine Liste aus allen Geschäfts-URLs machen (um später auf diese zuzugreiffen)

In [ ]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df_a["url"]:
    url_liste_ganz.append(url_anfang_original+element)
#diese werden zusammengefügt aus dem Anfang der Adresse sowie dem Geschäfts-URL

### Die einzelnen Geschäfte aufrufen und Infos rausholen

# Einschub: ich will speichern und mit diesen Daten dann arbeiten.

In [ ]:
#habe nun alle Seiten der Geschäfte auf meinem Rechner gespeichert. 
number = 0
for page in url_liste_ganz:
    page_content = requests.get(page)
    page_content = page_content.text
    with open("Geschaeft"+str(number)+".html", "w") as file:
        file.write(page_content)
        file.close()
    number +=1
    #Es sind nun exakt so viele Seiten auf meinem Rechner, wie das DataFrame oben Zeilen anzeigt

# Einschub-Ende. Ich habe es geschafft, das zu speichern. 

In [ ]:
#nun muss ich den Code ändern, sodass ich nicht mehr online zugreiffe sondern über meinen Ordner.
#hier nun die Liste der Zugriffe.
geschaefts_liste=[]#####ACHTUNG ich mache den RANGE kleiner, um weniger GEschäfte zu erhalten.

for seite_g in range(0,1963):
    gesch= "KRhtml/Geschaeft"+str(seite_g)+".html"
    geschaefts_liste.append(gesch)

In [ ]:
geschaefts_liste

In [ ]:
#Nun seite für Seite abrufen und in einem neuen Minidict unterbringen
#ACHTUNG: es bringt zumindest für einige die falschen Daten raus. und dann gleich mehrfach. 
#(Geschäft: 2129 8fach. Nur das richtige Datum ist nicht drinn.)
eingang_list=[]
geschaeft= "KRhtml/Geschaeft61.html"

file = open(geschaeft, 'r')
text = file.read()
soup_g = BeautifulSoup(text, 'html.parser')
tr_g_list=soup_g.find_all("tr")
td_g_list=tr_g_list[-1].find_all("td")#hier gehe ich zum Dokument. Dieses scheint von unten her an derselben Position zu sein
url_v_pdf = td_g_list[3].find("a")["href"] #nun hier die URL zum PDF des Vorstosses
gesch_nr = tr_g_list[-1].find_all("td")[0].text
einger_am = tr_g_list[1].find_all("td")[0].text   
ul_list = tr_g_list[2].find_all("ul")
li_list = ul_list[0].find_all("li")
    
for url in li_list:
    urlna = url.find("a")["href"]
    name = url.find("a").text
    
#So hole ich nun all die vorlagen Pdfs. Nur: wenn vier namen dort stehen, wird es auch vier Mal aufgelistet. 

    
    minidict_g={"GeschäftsNR":gesch_nr, "Einreichedatum":einger_am, "Namens-url": urlna, "Name eingereicht":name, "Link Vorstoss-PDF":url_v_pdf}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
    eingang_list.append(minidict_g)

### Hier versuche ich, ein separates Dataframe zu erstellen, mit VorlagenNR und url zum PDF der Vorlage
#### Da sonst jedes PDF vierfach angesteuert wird. 
Ich will so nun ein separates Dataframe generieren, in welchem ich dann auch die Einreichedaten und Namen aus den PDFs unterbringe. und es erst später dann mit den anderen Daten zusammenfüge. Sonst denke ich, gibts ein Puff.

In [ ]:
eingang_list=[]
geschaeft= "KRhtml/Geschaeft61.html"

file = open(geschaeft, 'r')
text = file.read()
soup_g = BeautifulSoup(text, 'html.parser')
tr_g_list=soup_g.find_all("tr")
td_g_list=tr_g_list[-1].find_all("td")#hier gehe ich zum Dokument. Dieses scheint von unten her an derselben Position zu sein
url_v_pdf = td_g_list[3].find("a")["href"] #nun hier die URL zum PDF des Vorstosses
gesch_nr = tr_g_list[-1].find_all("td")[0].text
einger_am = tr_g_list[1].find_all("td")[0].text  

In [ ]:
df_g=pd.DataFrame(eingang_list)

In [ ]:
df_g

In [ ]:
df_g=pd.DataFrame(eingang_list)
pd.set_option("display.max_rows", 100)

In [ ]:
#Das Datum muss noch als solches lesbar gemacht werden
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #verwandle string in datum

In [ ]:
df_mit_dat=pd.merge(df_a, df_g, how="left", on="GeschäftsNR") #ich verbinde die beiden DataFrames

In [ ]:
#nun will ich noch das Datum zum Idex machen
df_mit_dat.set_index("Einreichedatum", inplace=True)

In [ ]:
df_mit_dat

# Ein erstes Ergebnis = ab 2013 alle  Vorstösse

In [ ]:
#Nun will ich ein erstes Mal in die DAten schauen, zumindest ab 2013 - ob es ein Muster beim Einreichen gibt.
#DAzu krame ich jetzt alle Vorstösse raus
df_vo_all=df_mit_dat[{'Art des Geschäfts' : ["Motion","Interpellation","Postulat","Kleine Anfrage"] }] 

In [ ]:
df_vo_all["2013":].resample("SMS").count().plot(figsize=(8,4))

# Regex Baby! 
### Ich versuche mit Regex in Pandas an die Namen zu kommen

In [ ]:
#nun versuche ich mit Regex an die einzelnen Namen zu kommen.
regex_n=r"\bvon(.*und.\w*.\w*\b)"
regex_n2= r"^(\w*.\w*\b)\,.(\w*.\w*\b)"

In [ ]:
df_n1=df_a['Geschäft'].str.extract(regex_n)

In [ ]:
df_n1